In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm#
from bs4 import BeautifulSoup
from pathlib import Path
import sys
import os
import time

In [2]:
base_path = Path("../simulation_data/1_percent/v1")
graph_path = base_path / "network/graph.gexf"


In [3]:
graph = nx.read_gpickle(graph_path)

In [4]:
len(graph.nodes)

118820

In [5]:
mobility_events = base_path / "mobility/output_events.xml"

In [6]:
with open(mobility_events, "r") as handle:
    event_content = handle.read()

In [10]:
soup = BeautifulSoup(event_content, features="xml")

In [11]:
len(soup.find_all("event"))

22229336

In [13]:
from tqdm import tqdm
events = []
for event in tqdm(soup.find_all("event")):
    events.append(event)

100%|██████████| 22229336/22229336 [01:11<00:00, 309395.33it/s] 


In [23]:
dataframe_matrix = []
problems = []
for event in tqdm(soup.find_all("event")):
    try:
        dataframe_matrix.append([event["person"], event["time"], event["type"]])
    except:
        problems.append(event)

100%|██████████| 22229336/22229336 [05:02<00:00, 73401.13it/s] 


In [22]:
soup.find("event")

<event actType="work" facility="work_247522" link="614698" person="5279463" time="346.0" type="actend"/>

In [26]:
concept_dataframe = pd.DataFrame(dataframe_matrix, columns=["person_id", "timeframe", "action"])

In [74]:
concept_dataframe = pd.read_csv("concept_dataframe.csv")

In [75]:
from string import Template

class DeltaTemplate(Template):
    delimiter = "%"

def strfdelta(tdelta, fmt):
    d = {"D": tdelta.days}
    hours, rem = divmod(tdelta.seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    d["H"] = '{:02d}'.format(hours)
    d["M"] = '{:02d}'.format(minutes)
    d["S"] = '{:02d}'.format(seconds)
    t = DeltaTemplate(fmt)
    return t.substitute(**d)

def transform_date(x):
    try:
        return "2021-02-07 " + strfdelta(timedelta(seconds=float(x)), '%H-%M-%S')
    except:
        print(x)

tqdm.pandas()
timeframe_column = concept_dataframe["timeframe"].progress_apply(transform_date)

/usr/local/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 3640317/3640317 [00:39<00:00, 91688.35it/s]


In [76]:
import datetime
t = datetime.timedelta(seconds=float(concept_dataframe["timeframe"][0]))

In [77]:
concept_dataframe["timeframe"] = list(timeframe_column)

In [78]:
concept_dataframe.to_csv("concept_dataframe_v2.csv")

In [81]:
datetime.datetime.strptime("2021-02-07 00-05-46", '%Y-%m-%d %H-%M-%S')

datetime.datetime(2021, 2, 7, 0, 5, 46)

In [90]:
concept_dataframe = concept_dataframe.rename({"person_id": "id", "action": "concept"}, axis=1).drop("Unnamed: 0", axis=1)
concept_dataframe

,id,concept,datetime
0,5279463,actend,2021-02-07 00-05-46
1,5279463,departure,2021-02-07 00-05-46
2,5279463,PersonEntersVehicle,2021-02-07 00-05-46
3,5279463,vehicle enters traffic,2021-02-07 00-05-46
4,4356038,actend,2021-02-07 00-37-10
...,...,...,...
3640312,10861553,stuckAndAbort,2021-02-07 06-00-00
3640313,3089455,stuckAndAbort,2021-02-07 06-00-00
3640314,2475005,stuckAndAbort,2021-02-07 06-00-00
3640315,5045685,stuckAndAbort,2021-02-07 06-00-00


In [91]:
concept_dataframe.to_csv("concept_dataframe_v2.csv")

In [95]:
pd.to_datetime(concept_dataframe["datetime"], errors='coerce', format='%Y-%m-%d %H-%M-%S')

0         2021-02-07 00:05:46
1         2021-02-07 00:05:46
2         2021-02-07 00:05:46
3         2021-02-07 00:05:46
4         2021-02-07 00:37:10
                  ...        
3640312   2021-02-07 06:00:00
3640313   2021-02-07 06:00:00
3640314   2021-02-07 06:00:00
3640315   2021-02-07 06:00:00
3640316   2021-02-07 06:00:00
Name: datetime, Length: 3640317, dtype: datetime64[ns]